In [1]:
%load_ext autoreload
%autoreload 2

from os import path, listdir, getcwd
from src.credentials import Credentials as cr
import csv
import requests
import sys
import os
import src.split_datas as sd 
import pandas as pd

In [ ]:
sd.split_datas(cr.TITLE_BASICS, 'titleType')

In [ ]:
global_title_basics = pd.read_csv("./data/CURATED/titleType/movie.csv", usecols = ['tconst', 'originalTitle'])
global_title_basics = global_title_basics.dropna(axis = 0)
global_title_basics = global_title_basics.drop_duplicates()
global_title_basics.describe()

In [ ]:
sd.split_datas(cr.TITLE_PRINCIPALS, 'category')

In [ ]:
global_actor = pd.read_csv("./data/CURATED/category/actor.csv", usecols = ['tconst', 'nconst'])
global_actress = pd.read_csv("./data/CURATED/category/actress.csv", usecols = ['tconst', 'nconst'])
global_self = pd.read_csv("./data/CURATED/category/self.csv", usecols = ['tconst', 'nconst'])
global_title_principals = pd.concat([global_actor, global_actress, global_self])
global_title_principals = global_title_principals.dropna(axis = 0)
global_title_principals = global_title_principals.drop_duplicates()
global_title_principals.describe()

In [ ]:
global_name_basics = pd.read_csv("./data/RAW/name.basics.tsv", usecols = ['nconst', 'primaryName'], delimiter = '\t')
global_name_basics = global_name_basics.dropna(axis = 0)
global_name_basics = global_name_basics.drop_duplicates()
global_name_basics.describe()

In [ ]:
local_request_1 = global_title_principals.merge(global_title_basics, how = 'left', on = 'tconst')
local_request_1 = local_request_1.merge(global_name_basics, how = 'left', on = 'nconst')
local_request_1 = local_request_1.drop(['tconst', 'nconst'], axis = 1)
local_request_1 = local_request_1.dropna(axis = 0)
local_request_1_final = local_request_1.groupby('originalTitle').agg({'primaryName': ','.join}, axis = 0)

In [ ]:
local_request_1_final.to_csv('./data/REQUESTS/request_1.csv')